### Below notebook is for preprocessing to create data necessary for training Yolov8 model. For training a Yolov8 model, we need to make a config.yaml, which needs to contain paths for the train and valid dataset images and labels. Below is workflow for creating yolov8 .txt label files, which are essentially a list of x,y coordinate tuples for each class instance. Read Yolov8 documentations for more information.

In [2]:
#import necessary packages:
import numpy as np
import os
import torch
import pandas as pd
import cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import shutil
import random
from skimage.measure import label

In [5]:
final_df_src = r"\\fatherserverdw\Kevin\hubmap\obj_detect_bvonly.xlsx"
final_df = pd.read_excel(final_df_src)
final_df

,Unnamed: 0,id,type,coordinates,mask,rle,source_wsi,i,j
0,0,0033bbc76b6b,blood_vessel,"[[[169, 228], [168, 228], [167, 228], [166, 228], [165, 228], [164, 228], [163, 228], [163, 227], [162, 227], [161, 227], [161, 226], [160, 226], [160, 225], [159, 225], [159, 224], [158, 224], [1...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]],['73387 16 73897 28 74401 38 74911 43 75420 49 75931 52 76441 55 76952 57 77462 59 77973 62 78481 68 78991 71 79502 73 80013 78 80524 82 81036 86 81548 87 82059 89 82571 90 83083 90 83595 91 84107...,1,10240,43008
1,1,0033bbc76b6b,blood_vessel,"[[[1, 59], [0, 59], [0, 58], [0, 57], [0, 56], [0, 55], [0, 54], [0, 53], [0, 52], [0, 51], [0, 50], [0, 49], [0, 48], [0, 47], [0, 46], [0, 45], [0, 44], [0, 43], [0, 42], [0, 41], [0, 40], [0, 3...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]],['30 31 541 32 1053 31 1566 27 2079 25 2594 21 3109 15'],1,10240,43008
2,2,0033bbc76b6b,blood_vessel,"[[[406, 511], [405, 511], [404, 511], [403, 511], [402, 511], [401, 511], [400, 511], [399, 511], [398, 511], [397, 511], [396, 511], [395, 511], [394, 511], [393, 511], [392, 511], [391, 511], [3...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]],['190976 1 191485 4 191995 6 192506 7 193017 8 193528 9 194039 10 194551 10 195062 11 195574 11 196086 11 196597 12 197108 13 197620 13 198131 14 198642 15 199154 15 199665 16 200177 16 200689 16 ...,1,10240,43008
3,3,00656c6f2690,blood_vessel,"[[[511, 426], [511, 426], [510, 426], [510, 425], [509, 425], [509, 424], [508, 424], [508, 423], [507, 423], [507, 422], [506, 422], [506, 421], [505, 421], [505, 420], [505, 419], [504, 419], [5...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]],['257947 6 258457 12 258967 16 259478 18 259989 20 260500 22 261011 24 261522 26 262033 27'],1,10240,46080
4,4,00656c6f2690,blood_vessel,"[[[157, 404], [156, 404], [155, 404], [154, 404], [153, 404], [152, 404], [151, 404], [150, 404], [149, 404], [148, 404], [147, 404], [146, 404], [145, 404], [145, 403], [144, 403], [143, 403], [1...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]],['58240 8 58750 13 59261 15 59772 16 60284 17 60795 18 61306 20 61818 20 62329 21 62841 21 63352 22 63864 22 64376 22 64887 23 65399 24 65911 24 66422 25 66934 26 67446 26 67958 27 68470 27 68982 ...,1,10240,46080
...,...,...,...,...,...,...,...,...,...
3493,3493,fd2437954fd8,blood_vessel,"[[[481, 454], [480, 454], [479, 454], [478, 454], [477, 454], [476, 454], [475, 454], [474, 454], [473, 454], [473, 453], [472, 453], [471, 453], [471, 452], [470, 452], [469, 452], [469, 451], [4...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]],['238002 14 238510 20 239012 32 239520 37 240009 60 240515 67 241022 72 241532 75 242023 96 242533 99 243043 101 243541 115 244047 121 244551 129 245056 136 245558 146 246065 151 246575 153 247081...,1,5120,39424
3494,3494,fd2437954fd8,blood_vessel,"[[[416, 511], [415, 511], [414, 511], [413, 511], [412, 511], [411, 511], [410, 511], [409, 511], [408, 511], [407, 511], [406, 511], [405, 511], [404, 511], [403, 511], [402, 511], [401, 511], [4...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]],['191998 3 192502 11 193012 13 193522 15 194032 17 194542 19 195051 22 195561 24 196072 25 196584 25 197095 26 197606 27 198116 29 198625 32 199135 34 199645 36 200155 38 200665 40 201173 44 20167...,1,5120,39424
3495,3495,fd2437954fd8,blood_vessel,"[[[18, 362], [17, 362], [16, 362], [16, 361], [15, 361], [14, 361], [13, 361], [13, 360], [12, 360], [1

In [3]:
def set_seed(seed=42):
    np.random.seed(seed) #numpy specific random
    random.seed(seed) #python specific random (also for albumentation augmentations)
    torch.manual_seed(seed) #torch specific random
    torch.cuda.manual_seed(seed) #cuda specific random
    #when running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False #when deterministic = true, benchmark = False, otherwise might not be deterministic
    os.environ['PYTHONHASHSEED'] = str(seed)  #set a fixed value for the hash seed, for hashes like dictionary
set_seed(seed=42)

In [4]:
image_path_src = r"\\fatherserverdw\Kevin\hubmap\train_overlap\images"
image_full_paths = [os.path.join(image_path_src,x) for x in os.listdir(image_path_src) if x.endswith(".tif")]
bvmask_path_src = r"\\fatherserverdw\Kevin\hubmap\train_overlap\masks\blood_vessel"
bvmask_full_paths = [os.path.join(bvmask_path_src,x) for x in os.listdir(bvmask_path_src) if x.endswith(".tif")]
glmask_path_src = r"\\fatherserverdw\Kevin\hubmap\train_overlap\masks\glomerulus"
glmask_full_paths = [os.path.join(glmask_path_src,x) for x in os.listdir(glmask_path_src) if x.endswith(".tif")]

In [ ]:
bvmask_path = bvmask_full_paths[1]
bvmask = cv2.imread(bvmask_path,0)
bvmasks = label(bvmask)
num_masks = len(np.unique(bvmasks))
coords = []
types = []
for idx in range(1,num_masks):
    bin_mask = bvmasks == idx
    bin_mask = bin_mask.astype(np.uint8)
    contours, _ = cv2.findContours(bin_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_KCOS)
    coords.append(contours)
    types.append(0)

In [7]:
#need df of id, coordinate, and type
#coordinate and type list of lists, in same order, either 0 or 1, 0 = blood vessel, 1 = glomerulus for type.
ids = [os.path.basename(x) for x in image_full_paths]
data_df = pd.DataFrame(columns=["id","coords","type"])
data_df["id"] = ids
ids = data_df["id"].tolist()
for index,_id in tqdm(enumerate(ids),total = len(ids)):
    idcoords = []
    idtypes = []
    bvmask_path = os.path.join(bvmask_path_src,_id)
    bvmask = cv2.imread(bvmask_path,0)
    bvmasks = label(bvmask)
    num_bv_masks = len(np.unique(bvmasks))
    bvcoords = []
    bvtypes = []
    for idx in range(1,num_bv_masks):
        bvbin_mask = bvmasks == idx
        bvbin_mask = bvbin_mask.astype(np.uint8)
        bvcontours, _ = cv2.findContours(bvbin_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_KCOS)
        bvcoords.append(bvcontours)
        bvtypes.append(0)
    glmask_path = os.path.join(glmask_path_src,_id)
    if os.path.exists(glmask_path):
        glmask = cv2.imread(glmask_path,0)
        glmasks = label(glmask)
        num_gl_masks = len(np.unique(glmasks))
        for idx1 in range(1,num_gl_masks):
            glbin_mask = glmasks == idx1
            glbin_mask = glbin_mask.astype(np.uint8)
            glcontours, _ = cv2.findContours(bvbin_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_KCOS)
            bvcoords.append(glcontours)
            bvtypes.append(1)
    idcoords = bvcoords
    idtypes = bvtypes
    data_df["coords"].iloc[index] = idcoords
    data_df["type"].iloc[index] = idtypes

100%|██████████| 6874/6874 [04:22<00:00, 26.15it/s]


In [8]:
data_df

,id,coords,type
0,5631a47d5b0c.tif,"[([[[420 273]], [[416 277]], [[415 280]], [[41...",[0]
1,5631a47d5b0c_bot_right.tif,"[([[[164 17]], [[160 21]], [[159 24]], [[15...","[0, 0, 0, 0]"
2,2fd7649afbc1.tif,"[([[[395 291]], [[392 292]], [[389 293]], [[38...",[0]
3,2fd7649afbc1_top_right.tif,"[([[[164 17]], [[160 21]], [[159 24]], [[15...","[0, 0, 0, 0]"
4,88c95fb9fb14.tif,"[([[[495 243]], [[492 245]], [[489 248]], [[48...","[0, 0, 0]"
...,...,...,...
6869,83dd35ad9daf_bot_left.tif,"[([[[27 0]], [[30 1]], [[45 16]], [[47 16]],...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6870,dd4a8ba579c0.tif,"[([[[ 17 175]], [[ 15 176]], [[ 13 178]], [[ 1...","[0, 0, 1]"
6871,dd4a8ba579c0_top_left.tif,"[([[[27 0]], [[30 1]], [[45 16]], [[47 16]],...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6872,3b0d08f3e37a.tif,"[([[[115 54]], [[113 55]], [[108 58]], [[10...",[0]


In [9]:
#now make a function to create .txt file for the annotations for yolov8:
def create_label_txt_file(coordinate, type, ids, path):
    """
    :param coordinate: each coordinate array from df above
    :param type: each type array (same length as coordinates) from df above
    :param ids: each ids from df above
    :param path: save path
    :return: desired yolov8 yaml file
    """
    label_txt = ''
    for idx1 in range(len(coordinate)):
        indiv_type = type[idx1]
        if indiv_type == 0:
            label_txt += '0 '
        if indiv_type == 1:
            label_txt += '1 '
        indiv_coordinate = np.array(coordinate[idx1]).astype(float)
        indiv_coordinate /= float(512)
        coor_list = list(indiv_coordinate.reshape(-1).astype(str))
        coor_str = ' '.join(coor_list)
        #add string to label txt
        label_txt += f'{coor_str}\n'

    #write labels to txt file
    with open(f'{path}/{ids}.txt', 'w') as f:
        f.write(label_txt)

In [94]:
#create label .txt files for val:
label_save_path = r"\\fatherserverdw\Kevin\hubmap\yolov8_v4\all_labels"
num_images = data_df.shape[0]
coordinates = []
type_list = []
for idx2 in range(num_images):
    coord = data_df.iloc[idx2]["coords"]
    types = data_df.iloc[idx2]["type"]
    coordinates.append(coord)
    type_list.append(types)
for idx3 in tqdm(range(num_images)):
    create_label_txt_file(coordinate=coordinates[idx3], type=type_list[idx3], ids=data_df.iloc[idx3]["id"].split(".")[0],
                             path=label_save_path)

100%|██████████| 6874/6874 [03:36<00:00, 31.70it/s] 


### Now that the .txt file for all the labels have been created, we now want to divide the image-label pair into training and validation data.

In [12]:
data_df["real_id"] = [x.split('.tif')[0] if len(x.split('.tif')[0]) > 2 else x.split('_')[0] for x in data_df['id'].tolist()]
data_df["real_id"] = [x if len(x.split('_')[0]) < 1 else x.split('_')[0] for x in data_df['real_id'].tolist()]
data_df

,id,coords,type,real_id
0,5631a47d5b0c.tif,"[([[[420 273]], [[416 277]], [[415 280]], [[41...",[0],5631a47d5b0c
1,5631a47d5b0c_bot_right.tif,"[([[[164 17]], [[160 21]], [[159 24]], [[15...","[0, 0, 0, 0]",5631a47d5b0c
2,2fd7649afbc1.tif,"[([[[395 291]], [[392 292]], [[389 293]], [[38...",[0],2fd7649afbc1
3,2fd7649afbc1_top_right.tif,"[([[[164 17]], [[160 21]], [[159 24]], [[15...","[0, 0, 0, 0]",2fd7649afbc1
4,88c95fb9fb14.tif,"[([[[495 243]], [[492 245]], [[489 248]], [[48...","[0, 0, 0]",88c95fb9fb14
...,...,...,...,...
6869,83dd35ad9daf_bot_left.tif,"[([[[27 0]], [[30 1]], [[45 16]], [[47 16]],...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",83dd35ad9daf
6870,dd4a8ba579c0.tif,"[([[[ 17 175]], [[ 15 176]], [[ 13 178]], [[ 1...","[0, 0, 1]",dd4a8ba579c0
6871,dd4a8ba579c0_top_left.tif,"[([[[27 0]], [[30 1]], [[45 16]], [[47 16]],...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",dd4a8ba579c0
6872,3b0d08f3e37a.tif,"[([[[115 54]], [[113 55]], [[108 58]], [[10...",[0],3b0d08f3e37a


In [13]:
meta_df = pd.read_csv(r"\\fatherserverdw\Kevin\hubmap\tile_meta.csv")
meta_df = meta_df[meta_df["dataset"] < 3] # dataset 1, 2 only
meta_df = meta_df.rename(columns={'id': 'real_id'})
meta_df

,real_id,source_wsi,dataset,i,j
0,0006ff2aa7cd,2,2,16896,16420
2,00168d1b7522,2,2,14848,14884
4,0033bbc76b6b,1,1,10240,43008
5,003504460b3a,3,2,8192,11776
8,004daf1cbe75,3,2,6144,11264
...,...,...,...,...,...
7016,ff434af74304,4,2,3072,22528
7017,ff4897b3eda6,4,2,11776,20992
7021,ff66dec71c4c,3,2,5120,10752
7025,ff99cdef0f2a,4,2,5120,24064


In [14]:
final_data_df = pd.merge(data_df,meta_df,on='real_id')
final_data_df

,id,coords,type,real_id,source_wsi,dataset,i,j
0,5631a47d5b0c.tif,"[([[[420 273]], [[416 277]], [[415 280]], [[41...",[0],5631a47d5b0c,1,1,1536,42496
1,5631a47d5b0c_bot_right.tif,"[([[[164 17]], [[160 21]], [[159 24]], [[15...","[0, 0, 0, 0]",5631a47d5b0c,1,1,1536,42496
2,2fd7649afbc1.tif,"[([[[395 291]], [[392 292]], [[389 293]], [[38...",[0],2fd7649afbc1,1,1,1536,43008
3,2fd7649afbc1_top_right.tif,"[([[[164 17]], [[160 21]], [[159 24]], [[15...","[0, 0, 0, 0]",2fd7649afbc1,1,1,1536,43008
4,88c95fb9fb14.tif,"[([[[495 243]], [[492 245]], [[489 248]], [[48...","[0, 0, 0]",88c95fb9fb14,1,1,2048,39424
...,...,...,...,...,...,...,...,...
6869,83dd35ad9daf_bot_left.tif,"[([[[27 0]], [[30 1]], [[45 16]], [[47 16]],...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",83dd35ad9daf,4,2,13312,24576
6870,dd4a8ba579c0.tif,"[([[[ 17 175]], [[ 15 176]], [[ 13 178]], [[ 1...","[0, 0, 1]",dd4a8ba579c0,4,2,13312,25088
6871,dd4a8ba579c0_top_left.tif,"[([[[27 0]], [[30 1]], [[45 16]], [[47 16]],...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",dd4a8ba579c0,4,2,13312,25088
6872,3b0d08f3e37a.tif,"[([[[115 54]], [[113 55]], [[108 58]], [[10...",[0],3b0d08f3e37a,4,2,13312,26112


In [15]:
np.unique(final_data_df["source_wsi"], return_counts=True)

(array([1, 2, 3, 4], dtype=int64),
 array([2100, 1917, 1818, 1039], dtype=int64))

In [16]:
np.unique(final_data_df["dataset"], return_counts=True)

(array([1, 2], dtype=int64), array([1748, 5126], dtype=int64))

### Perform StratifiedKFold on the dataframe above, stratify according to dataset so equal ratio of each dataset (1 and 2) is in each fold. Let's do a 5-fold StratifiedKFold so we can perform 5-fold CV during training.

In [19]:
final_data_df_stratified = final_data_df.copy(deep=True)
strat_kfold = StratifiedKFold(shuffle = True, random_state = 42, n_splits = 5) #random_state for reproducibility

#split on white and non-white and add a new column fold to it:
for each_fold, (idx1,idx2) in enumerate (strat_kfold.split(X = final_data_df_stratified, y = final_data_df_stratified['dataset'])):
    final_data_df_stratified.loc[idx2,'fold'] = int(each_fold) #create new fold column with the fold number (up to 5)
final_data_df_stratified["fold"] = final_data_df_stratified["fold"].apply(lambda x: int(x)) #change fold to int, from 0~4
final_data_df_stratified

,id,coords,type,real_id,source_wsi,dataset,i,j,fold
0,5631a47d5b0c.tif,"[([[[420 273]], [[416 277]], [[415 280]], [[41...",[0],5631a47d5b0c,1,1,1536,42496,4
1,5631a47d5b0c_bot_right.tif,"[([[[164 17]], [[160 21]], [[159 24]], [[15...","[0, 0, 0, 0]",5631a47d5b0c,1,1,1536,42496,3
2,2fd7649afbc1.tif,"[([[[395 291]], [[392 292]], [[389 293]], [[38...",[0],2fd7649afbc1,1,1,1536,43008,1
3,2fd7649afbc1_top_right.tif,"[([[[164 17]], [[160 21]], [[159 24]], [[15...","[0, 0, 0, 0]",2fd7649afbc1,1,1,1536,43008,0
4,88c95fb9fb14.tif,"[([[[495 243]], [[492 245]], [[489 248]], [[48...","[0, 0, 0]",88c95fb9fb14,1,1,2048,39424,0
...,...,...,...,...,...,...,...,...,...
6869,83dd35ad9daf_bot_left.tif,"[([[[27 0]], [[30 1]], [[45 16]], [[47 16]],...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",83dd35ad9daf,4,2,13312,24576,0
6870,dd4a8ba579c0.tif,"[([[[ 17 175]], [[ 15 176]], [[ 13 178]], [[ 1...","[0, 0, 1]",dd4a8ba579c0,4,2,13312,25088,2
6871,dd4a8ba579c0_top_left.tif,"[([[[27 0]], [[30 1]], [[45 16]], [[47 16]],...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",dd4a8ba579c0,4,2,13312,25088,0
6872,3b0d08f3e37a.tif,"[([[[115 54]], [[113 55]], [[108 58]], [[10...",[0],3b0d08f3e37a,4,2,13312,26112,3


### Now save each dataframe and actually create dataset for each fold.

In [26]:
#for fold = 0
save_df_path = r"\\fatherserverdw\Kevin\hubmap\yolov8_v4"
val_df = final_data_df_stratified[final_data_df_stratified["fold"] == 0]
val_df = val_df.reset_index()
train_df = final_data_df_stratified[final_data_df_stratified["fold"] != 0]
train_df = train_df.reset_index()
save_dfs = False #already saved
if save_dfs:
    train_df.to_excel(os.path.join(save_df_path, "train_fold0.xlsx"))
    val_df.to_excel(os.path.join(save_df_path, "val_fold0.xlsx"))

In [29]:
real_img_src = r"\\fatherserverdw\Kevin\hubmap\yolov8\all_images"
real_label_src = r"\\fatherserverdw\Kevin\hubmap\yolov8\all_labels"
val_id_list = val_df["id"]
val_img_list = [os.path.join(real_img_src, x) for x in val_id_list]
val_label_list = [os.path.join(real_label_src, x.replace(".tif",".txt")) for x in val_id_list]
val_img_src = r"\\fatherserverdw\Kevin\hubmap\yolov8\val_fold0\images"
val_label_src = r"\\fatherserverdw\Kevin\hubmap\yolov8\val_fold0\labels"
for idx in tqdm(range(len(val_img_list))):
    val_img = val_img_list[idx]
    val_label = val_label_list[idx]
    shutil.copy(val_img, val_img_src)
    shutil.copy(val_label, val_label_src)

100%|██████████| 1374/1374 [02:41<00:00,  8.52it/s]


In [30]:
train_id_list = train_df["id"]
train_img_list = [os.path.join(real_img_src, x) for x in train_id_list]
train_label_list = [os.path.join(real_label_src, x.replace(".tif",".txt")) for x in train_id_list]
train_img_src = r"\\fatherserverdw\Kevin\hubmap\yolov8_v4\train_fold0\images"
train_label_src = r"\\fatherserverdw\Kevin\hubmap\yolov8_v4\train_fold0\labels"
for idx in tqdm(range(len(train_img_list))):
    train_img = train_img_list[idx]
    train_label = train_label_list[idx]
    shutil.copy(train_img, train_img_src)
    shutil.copy(train_label, train_label_src)

100%|██████████| 5500/5500 [09:52<00:00,  9.29it/s]
